In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

# Load JSON data
with open("../output/accounts_master.json", "r", encoding="utf-8") as f:
    data = json.load(f)

accounts = data["accounts"]
df = pd.json_normalize(accounts)

# Calculate manager counts
df["manager_count"] = df["page_managers"].apply(lambda x: sum(m['count'] for m in x) if isinstance(x, list) else 0)
df["unique_manager_countries"] = df["page_managers"].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Prepare data with clickable links
highly_managed = df[df["manager_count"] > 15][["name", "profile_url", "manager_count", "unique_manager_countries"]]
highly_managed["name"] = highly_managed.apply(
    lambda row: f"[{row['name']}]({row['profile_url']})", axis=1
)
highly_managed = highly_managed.drop(columns=["profile_url"])

# Summarize total by country
country_counts = {}
for managers in df["page_managers"].dropna():
    for manager in managers:
        country_counts[manager["country"]] = country_counts.get(manager["country"], 0) + manager["count"]

# Convert to DataFrame
country_df = pd.DataFrame(list(country_counts.items()), columns=["Country", "Manager Count"]).sort_values(by="Manager Count", ascending=False)

# Save chart
os.makedirs("report_images", exist_ok=True)
plt.figure(figsize=(10, 6))
sns.barplot(data=country_df.head(10), x="Manager Count", y="Country")
plt.title("Top 10 Countries by Total Manager Count")
plt.tight_layout()
chart_path = "report_images/high_manager_counts.png"
plt.savefig(chart_path)
plt.close()

# Save Markdown report
with open("../reports/high_manager_report.md", "w", encoding="utf-8") as report:
    report.write("# High Manager Count Analysis\n\n")
    report.write("## Pages with >15 Managers\n\n")
    report.write(highly_managed.to_markdown(index=False))
    report.write("\n\n## Top Countries by Manager Count\n\n")
    report.write(f"![high_manager_counts]({chart_path})\n")
